In [1]:
from profiler.core import *

In [2]:
pf = Profiler(workers=2, tol=0.01, eps=0.1, embedtxt=True)

In [3]:
pf.session.load_data(name='hospital', src=FILE, fpath='data/hospital.csv', check_param=True, na_values='empty')

INFO:profiler.utility:[0.007230997085571289] Load Data start

INFO:profiler.dataset:parameters used for data loading:
 {'na_values': 'empty', 'sep': ',', 'header': 'infer', 'dropcol': None, 'dropna': False, 'encoding': 'utf-8', 'normalize': True, 'min_categories_for_embedding': 10}
INFO:profiler.dataset:inferred types of attributes: {
    "ProviderNumber": "text",
    "HospitalName": "text",
    "Address1": "text",
    "City": "text",
    "State": "categorical",
    "ZipCode": "text",
    "CountyName": "text",
    "PhoneNumber": "text",
    "HospitalType": "text",
    "HospitalOwner": "text",
    "EmergencyService": "categorical",
    "Condition": "text",
    "MeasureCode": "text",
    "MeasureName": "text",
    "Score": "text",
    "Sample": "text",
    "Stateavg": "text"
}
INFO:profiler.dataset:(possible types: numeric, categorical, text, date)
INFO:profiler.dataset:inferred operators of attributes: {'ProviderNumber': ['equal'], 'HospitalName': ['equal'], 'Address1': ['equal'], 'City

In [4]:
pf.session.change_dtypes(['ProviderNumber', 'ZipCode', 'PhoneNumber', 'State', 'EmergencyService','Score', 'Sample'], 
                            [CATEGORICAL, NUMERIC, CATEGORICAL, TEXT, TEXT, NUMERIC, NUMERIC],
                            [None, None, None, None, None, r'(\d+)%', r'(\d+)\spatients'])

INFO:profiler.dataset:updated types of ProviderNumber to 'categorical'
INFO:profiler.dataset:updated operators of ProviderNumber to ['equal']
INFO:profiler.dataset:updated types of ZipCode to 'numeric'
INFO:profiler.dataset:updated operators of ZipCode to ['equal']
INFO:profiler.dataset:updated types of PhoneNumber to 'categorical'
INFO:profiler.dataset:updated operators of PhoneNumber to ['equal']
INFO:profiler.dataset:updated types of State to 'text'
INFO:profiler.dataset:updated operators of State to ['equal']
INFO:profiler.dataset:updated types of EmergencyService to 'text'
INFO:profiler.dataset:updated operators of EmergencyService to ['equal']
INFO:profiler.dataset:updated types of Score to 'numeric'
INFO:profiler.dataset:updated operators of Score to ['equal']
INFO:profiler.dataset:updated types of Sample to 'numeric'
INFO:profiler.dataset:updated operators of Sample to ['equal']
INFO:profiler.dataset:updated inferred operators of attributes: {'ProviderNumber': ['equal'], 'Hospi

In [5]:
pf.session.load_embedding(save=True, path='data/hospital/', load=True)

INFO:profiler.utility:[0.1036689281463623] Load Embedding start

INFO:profiler.utility:[0.1377577781677246] Load Embedding execution time: 0.034088850021362305



In [6]:
pf.session.load_training_data(multiplier = None)

INFO:profiler.utility:[0.14333891868591309] Create Training Data start

INFO:profiler.transformer:needs multiplier = 42 to bound the error in inv cov estimation <= 0.10000000
INFO:profiler.transformer:use multiplier = 42, and the bound is 0.00028699
INFO:profiler.transformer:Draw Pairs
100%|██████████| 17/17 [00:00<00:00, 259.26it/s]
INFO:profiler.transformer:Number of training samples: 50949
INFO:profiler.transformer:Computing Differences
INFO:profiler.transformer:estimated missing data probability in training data is 0.0300
INFO:profiler.utility:[1.4454398155212402] Create Training Data execution time: 1.3021008968353271



In [7]:
autoregress_matrix = pf.session.learn_structure(sparsity=0.05)

INFO:profiler.utility:[1.4520959854125977] Learn Structure start

INFO:profiler.learner:use threshold 0.0860
INFO:profiler.utility:[1.4731760025024414] Learn Structure execution time: 0.02108001708984375



In [8]:
inv_cov = pf.session.struct_engine.inv_cov

In [34]:
pf.session.struct_engine.param['visualize'] = False
G = pf.session.struct_engine.recover_moral_graphs(inv_cov)

In [35]:
order = []
while G.degrees.shape[0] > 0: 
    to_delete = G.degrees.index.values[G.degrees.degree.values.argmin()]
    order.append(to_delete)
    G.delete_node(to_delete)

In [36]:
order

[4, 8, 13, 14, 10, 11, 5, 9, 0, 1, 2, 3, 6, 7, 12, 15]

In [39]:
K = inv_cov.iloc[order, order]

In [38]:
# given the order, solve the matrix decommposition

In [47]:
import numpy as np
P = np.rot90(np.eye(K.shape[0]))

In [50]:
import pandas as pd
pd.DataFrame(data=np.dot(np.dot(P, K.values), P))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,11.534217,-6.685333,0.142966,0.141280,0.195506,0.159981,0.220367,0.185748,0.000000,0.000000,-0.222767,0.000000,0.000000,0.000000,0.000000,0.000000
1,-6.685333,11.401793,0.165754,0.196409,0.221765,0.183762,0.242405,0.210883,0.000000,0.000000,-0.328198,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.142966,0.165754,11.016757,-1.086019,-1.534921,-2.227304,-2.448756,-2.080999,-0.948226,-0.132225,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.141280,0.196409,-1.086019,9.242153,-2.277270,-1.182698,-1.446895,-1.270500,-0.807917,-0.364412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.195506,0.221765,-1.534921,-2.277270,10.135272,-1.569389,-1.727117,-1.514814,-0.488059,-0.320404,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.159981,0.183762,-2.227304,-1.182698,-1.569389,11.203885,-2.478843,-2.248733,-0.800040,-0.140159,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.220367,0.242405,-2.448756,-1.446895,-1.727117,-2.478843,12.157388,-2.578825,-0.927495,-0.315591,0.000000,-0.160336,0.000000,0.000000,0.000000,0.000000
7,0.185748,0.210883,-2.080999,-1.270500,-1.514814,-2.248733,-2.578825,11.250367,-0.862046,-0.164365,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,-0.948226,-0.807917,-0.488059,-0.800040,-0.927495,-0.862046,6.870316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,-0.132225,-0.364412,-0.320404,-0.140159,-0.315591,-0.164365,0.000000,6.013206,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [57]:
PAP = np.dot(np.dot(P, K.values), P.transpose())

In [58]:
from sksparse.cholmod import cholesky, analyze
from scipy import sparse
A = sparse.csc_matrix(PAP)
factor = cholesky(A)

In [59]:
L = factor.L_D()[0].toarray()

In [65]:
data = pd.DataFrame(np.eye(K.shape[0])- np.dot(np.dot(P, L), P.transpose()), columns=K.columns)
data.index = K.columns

In [66]:
data

,State,HospitalType,Score,Sample,EmergencyService,Condition,ZipCode,HospitalOwner,ProviderNumber,HospitalName,Address1,City,CountyName,PhoneNumber,MeasureCode,Stateavg
State,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
HospitalType,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Score,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Sample,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
EmergencyService,0.0,0.0,0.0,0.0,0.0,0.169726,0.519361,0.567632,0.364330,0.256830,0.190723,0.118857,-0.033305,-0.012408,0.000000,0.000000
Condition,0.0,0.0,0.0,0.0,0.0,0.000000,0.049924,0.067905,0.051488,0.025155,0.043037,0.039481,0.000000,0.000000,0.000000,0.000000
ZipCode,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.232312,0.143415,0.097538,0.072132,0.087531,0.000000,0.000000,0.000000,0.000000
HospitalOwner,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.382066,0.261976,0.193903,0.139183,-0.042674,-0.016121,0.000000,0.000000
ProviderNumber,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.290740,0.221092,0.158549,-0.049586,-0.019126,0.000000,0.020845
HospitalName,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.197042,0.129466,-0.037039,-0.013885,0.000000,0.000000
